In [1]:
import numpy as np
import pandas as pd
import openpyxl
import os 
import xlrd
# Setting Directory
os.chdir('C:/Users/TAM/Documents/Vu Quang Nguyen/VACP/')

In [2]:
# NHAP - XUAT - TON  * GIA BAN * DANH MUC HANG HOA

data_nhapxuatton=pd.read_excel("../VACP/Python NCC Min/Tổng hợp Nhập - Xuất - Tồn YTD.xls",usecols="A:B,G:O",skipfooter=1,header=5)
data_nhapxuatton=data_nhapxuatton.rename(columns={
                                "Unnamed: 0":"ID",
                                "Unnamed: 1":"Tên vật tư",
                                "Số lượng":"Tổng đầu kỳ", 
                                "Giá trị":"Giá trị đầu kỳ",
                                "Số lượng.1":"Tổng nhập", 
                                "Giá trị.1":"Giá trị nhập",
                                "Số lượng.2":"Tổng xuất", 
                                "Giá trị.2":"Giá trị xuất",
                                "Số lượng.3":"Tổng tồn", 
                                "Giá trị.3":"Giá trị tồn" }).dropna(subset=['ID']).reset_index(drop=True)


data_banhang=pd.read_excel("../VACP/Bao cao Ban hang Power BI/Sổ chi tiết bán hàng.xls",usecols="A,C:D,F,H,K:M",skipfooter=1,header=6)
data_banhang=data_banhang.rename(columns={
                                "Unnamed: 0":"Ngày bán",
                                "Unnamed: 2":"ID Khách hàng",
                                "Unnamed: 3":"Hãng",
                                "Unnamed: 5":"ID",
                                "Unnamed: 7":"Tên hàng",
                                "Số lượng":"Tổng bán", 
                                "Đơn giá gốc":"Giá bán",
                                "Giá bán":"Giá bán giảm"
                                }).dropna(subset=['ID']).reset_index(drop=True)
                                
data_giaban=pd.read_excel("../VACP/Python NCC Min\Báo cáo giá bán, tồn kho.xls",usecols="C,F",skipfooter=1,header=4)
data_giaban=data_giaban.rename(columns={"Mã số":"ID"}).dropna(subset=['ID']).reset_index(drop=True)

data_danhmuchanghoa=pd.read_excel("../VACP/Python NCC Min/Danh mục vật tư hàng hóa .xls",usecols="A,C:D,M")
data_danhmuchanghoa=data_danhmuchanghoa.rename(columns={
                                "Unnamed: 0":"ID Nhóm hàng",
                                "Unnamed: 2":"ID",
                                "Unnamed: 3":"Tên vật tư",
                                "Unnamed: 12":"Hãng"
                                }).dropna(subset=['ID','ID Nhóm hàng']).reset_index(drop=True)



C:\Users\TAM\AppData\Local\Temp\ipykernel_12860\773044771.py:3: FutureWarning: Defining usecols with out of bounds indices is deprecated and will raise a ParserError in a future version.
  data_nhapxuatton=pd.read_excel("../VACP/Python NCC Min/Tổng hợp Nhập - Xuất - Tồn YTD.xls",usecols="A:B,G:O",skipfooter=1,header=5)


In [4]:
#Bán hàng gần đây
max_date_banhang=data_banhang.groupby(["ID"])["Ngày bán"].max()
data_banhang_latest=data_banhang.merge(max_date_banhang,on="ID",how="left",suffixes=('', '_max'))
data_banhang_latest=data_banhang_latest[data_banhang_latest["Ngày bán"]==data_banhang_latest["Ngày bán_max"]].drop('Ngày bán_max',axis=1).drop_duplicates(subset="ID",keep='first').reset_index(drop=True)
 

In [35]:
# Gộp dữ liệu
data_nhap_ban=data_nhapxuatton.merge(data_danhmuchanghoa[["ID","Hãng"]],on="ID").merge(data_banhang_latest[["ID","Ngày bán","Giá bán giảm"]],on='ID').merge(data_giaban,on="ID")


# Tỷ lệ hàng được xuất bán
data_nhap_ban["Tỷ lệ xuất hàng"] = (data_nhap_ban["Tổng xuất"])/(data_nhap_ban["Tổng xuất"]+data_nhap_ban["Tổng tồn"])

# TÍnh số tháng tồn trong kho
data_nhap_ban["Số tháng tồn trong kho"]=(363/30)/data_nhap_ban["Giá vốn"]/(np.where(
    # Nếu Tồn dương và đầu kỳ dương thì lấy Giá vốn / (TB Giá trị đầu kỳ + Cuối kỳ)
                    (data_nhap_ban["Giá trị đầu kỳ"]>0) & (data_nhap_ban["Giá trị tồn"]>0),
                               np.mean([data_nhap_ban["Tổng đầu kỳ"],data_nhap_ban["Tổng tồn"]])/
                                np.mean([data_nhap_ban["Giá trị đầu kỳ"],data_nhap_ban["Giá trị tồn"]]),
                                     np.where((data_nhap_ban["Giá trị nhập"]>0) & (data_nhap_ban["Giá trị tồn"]>0),
                                              np.mean([data_nhap_ban["Tổng nhập"],data_nhap_ban["Tổng tồn"]])/
                                              np.mean([data_nhap_ban["Giá trị nhập"],data_nhap_ban["Giá trị tồn"]]),-999
                                )))

# Show kết quả
data_nhap_ban_filter=data_nhap_ban[["ID","Tên vật tư","Hãng","Giá vốn","Giá bán","Tổng đầu kỳ","Tổng nhập","Tổng xuất","Tổng tồn", "Giá trị đầu kỳ", "Giá trị nhập" ,"Giá trị xuất", "Giá trị tồn","Số tháng tồn trong kho","Tỷ lệ xuất hàng"]]
data_nhap_ban_filter=data_nhap_ban_filter.query("`Giá vốn`>0 & `Tổng tồn` >0 ").sort_values(by=["Số tháng tồn trong kho"], ascending=False).reset_index(drop=True)

In [20]:
data_nhap_ban_filter.groupby(["ID","Hãng"]).agg(
   {
     'Tổng tồn':'sum',
     'Số tháng tồn trong kho': 'sum'
   }
)

,,Tổng tồn,Số tháng tồn trong kho
ID,Hãng,,
016-060-0008,MEYLE,1,9.897507
016-060-0009,MEYLE,1,9.979531
04009-86175,TOYOTA,41,5.155739
04111-0C210,TOYOTA,16,1.339760
04111-0P065,TOYOTA,2,0.834546
...,...,...,...
ZA-50TKB3504B1R,NSK,104,15.321407
ZA-54KWH01R-Y-2CA-01,NSK,20,4.530050
ZA-54KWH02D-Y-2CA-01,NSK,6,4.484186


In [37]:
data_nhap_ban_filter

,ID,Tên vật tư,Hãng,Giá vốn,Giá bán,Tổng đầu kỳ,Tổng nhập,Tổng xuất,Tổng tồn,Giá trị đầu kỳ,Giá trị nhập,Giá trị xuất,Giá trị tồn,Số tháng tồn trong kho,Tỷ lệ xuất hàng
0,90913-02098,Ghít 8-13 Toyota (8),TOYOTA,6901.103125,16000.0,384,0,144,240,2523820.0,0,946437.0,1577383.0,5.496541e+02,0.375000
1,90913-02081,Ghít 6-11,TOYOTA,7433.814706,16000.0,256,0,136,120,1812436.0,0,962856.0,849580.0,5.102656e+02,0.531250
2,90913-02093,Ghít 5-9.5 Toyota (16),TOYOTA,8315.931392,16000.0,60,326,352,34,687724.0,2369366,2787817.0,269273.0,4.561389e+02,0.911917
3,90913-02101,Ghít 5-10 Toyota,TOYOTA,8533.084994,16000.0,80,2040,1676,444,490962.0,16737729,13620429.0,3608262.0,4.445309e+02,0.790566
4,90080-38070,Cao su ắc nhíp,TOYOTA,10282.539750,30000.0,302,900,800,402,2867033.0,8909742,7834316.0,3942459.0,3.688991e+02,0.665557
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1108,SL-3890L,Cân bằng trước Prado LH,555,314214.627794,320000.0,362,12,340,34,97456441.0,3591024,101745689.0,-698224.0,-3.854726e-08,0.909091
1109,SE-T831L,Lái ngoài LH,555,278484.097500,290000.0,20,1,20,1,4822637.0,265223,5304459.0,-216599.0,-4.349301e-08,0.952381
1110,SE-T831R,Lái ngoài RH,555,278484.097500,290000.0,20,1,20,1,4822637.0,265223,5304459.0,-216599.0,-4.349301e-08,0.952381
1111,SE-3753L,Lái ngoài Camry 07 LH,555,226904.545361,290000.0,26,72,97,1,5775441.0,14512816,20961658.0,-673401.0,-5.337977e-08,0.989796


In [22]:
data_MITSUBISHI_6M=data_nhap_ban_filter.query(
        "`Hãng` == 'MITSUBISHI' & `Số tháng tồn trong kho`>6").sort_values(
                by="Số tháng tồn trong kho",ascending=False).reset_index(drop=True)
data_MITSUBISHI_6M.head()


,ID,Tên vật tư,Hãng,Giá vốn,Giá bán,Tổng đầu kỳ,Tổng nhập,Tổng xuất,Tổng tồn,Giá trị đầu kỳ,Giá trị nhập,Giá trị xuất,Giá trị tồn,Số tháng tồn trong kho
0,MR150094,Cao su cân bằng trước lỗ 30,MITSUBISHI,27729.450000,45000.0,6,0,2,4,158454.0,0,52818.0,105636.0,136.793899
1,MB338595,Cao su cân bằng trước lỗ 24,MITSUBISHI,39371.325000,45000.0,12,2,4,10,449959.0,74993,149986.0,374966.0,96.344727
2,1370B272,Cút nước trên Xpander 19-21,MITSUBISHI,65332.050000,95000.0,3,0,1,2,186664.0,0,62221.0,124443.0,58.060624
3,8355A092,Đèn phản quang badersoc sau RH Xpander 19-21,MITSUBISHI,101600.316176,110000.0,39,20,34,25,3743690.0,1954992,3289915.0,2408767.0,37.334722
4,8355A091,Đèn phản quang badersoc sau LH Xpander 19-21,MITSUBISHI,101613.431818,110000.0,39,21,33,27,3743690.0,2051343,3193565.0,2601468.0,37.329903


In [39]:
# EXPORT FILE DATA
with pd.ExcelWriter('../VACP/Lọc tồn theo hãng/Export.xlsx') as writer:
    # data_mitsubishi.to_excel(writer, sheet_name='MITSUBISHI')
    data_nhap_ban.to_excel(writer,sheet_name="Nhap Ban")
    data_nhap_ban_filter.to_excel(writer,sheet_name="Filter")
    data_MITSUBISHI_6M.to_excel(writer,sheet_name="MITSUBISHI_6Thang")